In [12]:
# From here: https://stackoverflow.com/questions/49264194/import-py-file-in-another-directory-in-jupyter-notebook
import sys
import os
from datetime import datetime

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import numpy as np
import copy

import src.utils as utils
from src.nqs import RBM, IsingHamiltonian, IsingHamiltonianReduced
from config_parser import get_config_file

import matplotlib.pyplot as plt
%matplotlib notebook


ModuleNotFoundError: No module named 'src'

In [25]:
visible_size = 4
hidden_size = 6
seed = 44  # Seed for random number generator
walker_steps = 1000
gradient_steps = 100


In [26]:
np.random.seed(seed)

gamma = utils.random_gamma(visible_size)
ih = IsingHamiltonian(gamma=gamma)
ihr = IsingHamiltonianReduced(gamma=gamma)


In [27]:
eig,_ = np.linalg.eigh(ih)
I_truth = np.min(eig)
print(f"E_truth: {I_truth}")


E_truth: -3.313112070611122


In [28]:
ising_reduced_rbm = RBM(visible_size=visible_size, hidden_size=hidden_size, hamiltonian=ihr, walker_steps=walker_steps)
ir_energy_list = [it for it in ising_reduced_rbm.train(iterations=gradient_steps, lr=0.01, print_energy=False)]

func:train args:[(<src.nqs.RBM object at 0x000001D5B6D40FA0>,), {'iterations': 100, 'lr': 0.01, 'print_energy': False}] took: 35.90792655944824 sec


In [29]:
ising_rbm = RBM(visible_size=visible_size, hidden_size=hidden_size, hamiltonian=ih, walker_steps=walker_steps)
i_energy_list = [it for it in ising_rbm.train(iterations=gradient_steps, lr=0.01,print_energy=False)]

func:train args:[(<src.nqs.RBM object at 0x000001D5B6A93C70>,), {'iterations': 100, 'lr': 0.01, 'print_energy': False}] took: 35.41229557991028 sec


In [30]:
plt.axhline(y=I_truth, color='k', linestyle='--')
plt.plot(i_energy_list)
plt.plot(ir_energy_list, 'r')
plt.grid()

plt.legend(['Truth', 'Ising', 'Ising Reduced'])

plt.show()

In [8]:
time_matrix = []
time_reduced = []
n_list = []


for i in range(4, 11):
    visible_size = i
    hidden_size = 2*i
    
    n_list.append(visible_size)
    
    gamma = utils.random_gamma(visible_size)
    ih = IsingHamiltonian(gamma=gamma)
    ihr = IsingHamiltonianReduced(gamma=gamma)
    
    ising_rbm = RBM(visible_size=visible_size, hidden_size=hidden_size, hamiltonian=ih, walker_steps=0)
    ising_rbm.train(iterations=gradient_steps, lr=0.01,print_energy=False)
    time_matrix.append(ising_rbm.train.run_time)

    ising_reduced_rbm = RBM(visible_size=visible_size, hidden_size=hidden_size, hamiltonian=ihr, walker_steps=walker_steps)
    ising_reduced_rbm.train(iterations=gradient_steps, lr=0.01,print_energy=False)
    time_reduced.append(ising_reduced_rbm.train.run_time)
    

func:train args:[(<src.nqs.RBM object at 0x000001D5B325B0A0>,), {'iterations': 100, 'lr': 0.01, 'print_energy': False}] took: 0.23652148246765137 sec
func:train args:[(<src.nqs.RBM object at 0x000001D5B317B460>,), {'iterations': 100, 'lr': 0.01, 'print_energy': False}] took: 43.81325602531433 sec
func:train args:[(<src.nqs.RBM object at 0x000001D5B3350E20>,), {'iterations': 100, 'lr': 0.01, 'print_energy': False}] took: 1.2047710418701172 sec
func:train args:[(<src.nqs.RBM object at 0x000001D5B3350370>,), {'iterations': 100, 'lr': 0.01, 'print_energy': False}] took: 54.8105583190918 sec
func:train args:[(<src.nqs.RBM object at 0x000001D5B33B4F70>,), {'iterations': 100, 'lr': 0.01, 'print_energy': False}] took: 7.19272780418396 sec
func:train args:[(<src.nqs.RBM object at 0x000001D5B0724310>,), {'iterations': 100, 'lr': 0.01, 'print_energy': False}] took: 77.43177437782288 sec
func:train args:[(<src.nqs.RBM object at 0x000001D5B3350370>,), {'iterations': 100, 'lr': 0.01, 'print_energy':

In [1]:
plt.figure(2)

plt.semilogy(n_list, time_matrix)
plt.semilogy(n_list, time_reduced)
plt.grid()

plt.title("Training time using matrix and reduced Hamiltonian representations")
plt.legend(['Matrix Hamiltonian', 'Reduced Hamiltonian'])
plt.xlabel('# visible nodes')
plt.ylabel('time (s)')

NameError: name 'plt' is not defined

In [33]:
# Save figure as pdf
username = os.getlogin()
desktop_path = f"{os.path.expanduser(os.getenv('USERPROFILE'))}\\Desktop"

date_format = '%Y.%m.%d;%H%M'

path = f"{desktop_path}\\{datetime.now().strftime(date_format)}-Ising timing"
os.makedirs(path)
plt.savefig(f"{path}/ising_time.pdf")